In [6]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

from tensorflow.keras import datasets, layers, models

In [7]:
import os
import pandas as pd

# Define paths to your training and testing directories
train_dir = 'Datasets/RAF-FER-SFEW/train'  # Replace with your training directory path
test_dir = 'Datasets/RAF-FER-SFEW/test'    # Replace with your testing directory path

# Function to add images from a directory to a list
def process_directory(directory, data_list):
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)

        # Check if it's a directory
        if os.path.isdir(class_dir):
            # Loop through each image in the folder
            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                # Append to the data list
                data_list.append({'filepath': image_path, 'label': class_name})

# Initialize an empty list for storing data
data_list = []

# Add training images to the data list
process_directory(train_dir, data_list)

# Add testing images to the data list
process_directory(test_dir, data_list)

# Create a DataFrame from the list
df = pd.DataFrame(data_list)

# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

# print 15 random samples of the DataFrame
df.sample(15)

,filepath,label
22457,Datasets/RAF-FER-SFEW/train/disgust/train_0543...,disgust
44080,Datasets/RAF-FER-SFEW/train/happy/train_03946_...,happy
5516,Datasets/RAF-FER-SFEW/test/happy/PrivateTest_6...,happy
48241,Datasets/RAF-FER-SFEW/train/angry/Training_161...,angry
48214,Datasets/RAF-FER-SFEW/train/neutral/Training_9...,neutral
44811,Datasets/RAF-FER-SFEW/test/happy/PublicTest_14...,happy
45062,Datasets/RAF-FER-SFEW/train/surprise/Training_...,surprise
5300,Datasets/RAF-FER-SFEW/train/fear/Training_3563...,fear
26121,Datasets/RAF-FER-SFEW/test/happy/test_1143_ali...,happy
16031,Datasets/RAF-FER-SFEW/test/angry/PublicTest_38...,angry


In [8]:
# Define paths to your training and testing directories
# train_dir = 'Datasets/RAF-DB/DATASET/train'
# test_dir = 'Datasets/RAF-DB/DATASET/test'
# train_dir = 'Datasets/FER2013/train'
# test_dir = 'Datasets/FER2013/test'
# Set the image size and batch size
image_size = (96, 96) # Can be increased to improve accuracy or decreased to improve speed. (48, 48) for FER2013, (224, 224) for RAF-DB
batch_size = 64

# Split the data into training, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)  # 80% training, 20% test
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42)  # Of the 80% training, split into 60% training and 20% validation

# Calculate and print split ratios
total_samples = len(df)
train_ratio = len(train_df) / total_samples
val_ratio = len(val_df) / total_samples
test_ratio = len(test_df) / total_samples

print(f"Total samples: {total_samples}")
print(f"Training set: {train_ratio:.2f} ({len(train_df)} samples)")
print(f"Validation set: {val_ratio:.2f} ({len(val_df)} samples)")
print(f"Test set: {test_ratio:.2f} ({len(test_df)} samples)\n")

# Create an ImageDataGenerator for data augmentation (optional)
train_datagen = ImageDataGenerator(
    rescale=1./255, # Normalize pixel values to [0, 1]
    rotation_range=15,  # rotation
    width_shift_range=0.05, # horizontal shift (only 5% since faces are centered)
    height_shift_range=0.05, # vertical shift (only 5% since faces are centered)
    shear_range=0.1, 
    # zoom_range=0.1,   zoom (with current dataset not needed, since faces are centered)
    horizontal_flip=True, # flip images horizontally
    fill_mode='constant', # fill in missing pixels (nearest / constant)
    # brightness_range=[0.8, 1.2] # darken and lighten images
)

val_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for validation data
test_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for test data

# Create generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False
)

# # Compute class weights
# class_weights = compute_class_weight(
#     class_weight='balanced',
#     classes=np.unique(train_generator.classes),
#     y=train_generator.classes
# )
# 
# class_weights_dict = dict(enumerate(class_weights))
# print(class_weights_dict)

# # Compute class weights
# labels = train_df['label'].values
# unique_classes = np.unique(labels)
# class_weights = compute_class_weight('balanced', classes=unique_classes, y=labels)
# class_weights_dict = {unique_classes[i]: weight for i, weight in enumerate(class_weights)}

# Assuming 'labels' contains your class labels for the training data
labels = train_df['label'].values
unique_classes = np.unique(labels)
# Compute class weights for balanced training
class_weights = compute_class_weight('balanced', classes=unique_classes, y=labels)
# Get class indices from the generator
class_indices = train_generator.class_indices
# Ensure the order of `unique_classes` matches the order in `class_indices`
ordered_unique_classes = sorted(unique_classes, key=lambda x: class_indices[x])
# Create a dictionary mapping class indices to their weights
class_weights_dict = {class_indices[label]: weight for label, weight in zip(ordered_unique_classes, class_weights)}

Total samples: 52490
Training set: 0.60 (31494 samples)
Validation set: 0.20 (10498 samples)
Test set: 0.20 (10498 samples)

Found 31494 validated image filenames belonging to 7 classes.
Found 10498 validated image filenames belonging to 7 classes.
Found 10498 validated image filenames belonging to 7 classes.


In [9]:
# Create a non-preprocessing ImageDataGenerator
no_preprocessing_datagen = ImageDataGenerator()

# Create a temporary generator to fetch a batch of original images
temp_generator = no_preprocessing_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Keep the same order as the original generator
)

# Function to plot images in a grid
def plot_images(images_arr):
    fig, axes = plt.subplots(5, 5, figsize=(12, 12))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img.astype('uint8'))  # Cast to uint8 for correct image display
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# Fetch a batch of original images
original_batch = next(temp_generator)
original_images = original_batch[0][:25]  # Select first 25 images

# Plot the original images
# plot_images(original_images)

Found 41839 images belonging to 7 classes.


In [10]:
# Function to plot images in a grid
def plot_images(images_arr):
    fig, axes = plt.subplots(5, 5, figsize=(12, 12))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img, cmap='gray')  # Set the colormap to 'gray'
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# Get a batch of images
example_batch = next(train_generator)
example_images = example_batch[0][:25]  # Select first 25 imaages

# Plot the images
# plot_images(example_images)

In [16]:
dropProb=0.0 # fraction of units to drop
nfilter_1 = 256
nfilter_2 = 128
nfilter_3 = 128
nfilter_4 = 64
nhid=64

model = models.Sequential()
model.add(layers.Conv2D(filters=nfilter_1, kernel_size=(3, 3), 
                        activation='relu', input_shape=(96, 96, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(rate=dropProb))
model.add(layers.Conv2D(filters=nfilter_2, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(rate=dropProb))
model.add(layers.Conv2D(filters=nfilter_3, kernel_size=(3, 3), activation='relu'))

model.add(layers.Dropout(rate=dropProb))
model.add(layers.Conv2D(filters=nfilter_3, kernel_size=(3, 3), activation='relu'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 94, 94, 256)       2560      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 47, 47, 256)       0         
 g2D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 47, 47, 256)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 45, 45, 128)       295040    
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 22, 22, 128)       0         
 g2D)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 22, 22, 128)      

In [17]:
# Early stopping to prevent overfitting. This stops training when the model's performance on the validation set starts to degrade.
early_stopper = EarlyStopping(
    monitor='val_loss',  # Metric to be monitored
    patience=3,         # Number of epochs with no improvement after which training will be stopped. Reduced from 10
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored metric
)

# Create a ModelCheckpoint callback
checkpoint = ModelCheckpoint(
    'model_checkpoint.keras',  # Path where to save the model
    monitor='val_loss',     # Metric to monitor
    save_best_only=False,    # Save only the best model. Set False to save the model at the end of every epoch so restarting from specific epoch is possible
    save_weights_only=False, # Save only the weights
    mode='min',             # Minimize the monitored metric (val_loss) min before
    verbose=1               # Verbose output
)

In [18]:
# Load the last saved weights
#model.load_weights('model_checkpoint.keras')

epochs = 30 # When resuming training, set epochs to the total number of epochs you want to train, not just the additional epochs. The model.fit() method continues training for the specified number of epochs, starting from the current epoch count. Typically, 10 to 100 epochs are used. Start with e.g. 30-50.

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    class_weight=class_weights_dict,
    callbacks=[early_stopper, checkpoint]
)

# Save the training history for later analysis
with open('training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

import os
import pandas as pd

# Define paths to your training and testing directories
train_dir = 'Datasets/RAF-FER-SFEW/train'
test_dir = 'Datasets/RAF-FER-SFEW/test'

# Function to add images from a directory to a list
def process_directory(directory, data_list):
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                data_list.append({'filepath': image_path, 'label': class_name})

data_list = []
process_directory(train_dir, data_list)
process_directory(test_dir, data_list)
df = pd.DataFrame(data_list)
df = df.sample(frac=1).reset_index(drop=True)

image_size = (96, 96)
batch_size = 64

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='constant',
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False
)

labels = train_df['label'].values
unique_classes = np.unique(labels)
class_weights = compute_class_weight('balanced', classes=unique_classes, y=labels)
class_indices = train_generator.class_indices
ordered_unique_classes = sorted(unique_classes, key=lambda x: class_indices[x])
class_weights_dict = {class_indices[label]: weight for label, weight in zip(ordered_unique_classes, class_weights)}

# Model Definition
dropProb = 0.5  # Adjust dropout rate as needed
nfilter_1 = 64
nfilter_2 = 128
nfilter_3 = 256
nfilter_4 = 512
nhid = 64

model = Sequential([
    Conv2D(nfilter_1, (3, 3), activation='relu', input_shape=(96, 96, 1)),
    MaxPooling2D(2, 2),
    Dropout(dropProb),
    Conv2D(nfilter_2, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(dropProb),
    Conv2D(nfilter_3, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(dropProb),
    Flatten(),
    Dense(nhid, activation='relu'),
    Dropout(dropProb),
    Dense(len(unique_classes), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Callbacks
early_stopper = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('model_checkpoint.keras', save_best_only=True, monitor='val_loss', mode='min')

epochs = 3
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    class_weight=class_weights_dict,
    callbacks=[early_stopper, checkpoint]
)

# Save history
with open('training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)


Found 31494 validated image filenames belonging to 7 classes.
Found 10498 validated image filenames belonging to 7 classes.
Found 10498 validated image filenames belonging to 7 classes.
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 94, 94, 64)        640       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 47, 47, 64)        0         
 g2D)                                                            
                                                                 
 dropout_13 (Dropout)        (None, 47, 47, 64)        0         
                                                                 
 conv2d_16 (Conv2D)          (None, 45, 45, 128)       73856     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 22, 22, 128)       0         


In [25]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
import pandas as pd

# Define paths to your training and testing directories
train_dir = 'Datasets/RAF-FER-SFEW/train'
test_dir = 'Datasets/RAF-FER-SFEW/test'

def process_directory(directory, data_list):
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                data_list.append({'filepath': image_path, 'label': class_name})

data_list = []
process_directory(train_dir, data_list)
process_directory(test_dir, data_list)
df = pd.DataFrame(data_list)

min_count = df['label'].value_counts().min()
balanced_df = pd.concat([
    df[df['label'] == label].sample(min_count, random_state=42) for label in df['label'].unique()
])
balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

image_size = (96, 96)
batch_size = 32

train_df, test_df = train_test_split(balanced_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(train_df, x_col='filepath', y_col='label',
                                                    target_size=image_size, batch_size=batch_size,
                                                    color_mode='rgb', class_mode='categorical')

val_generator = val_datagen.flow_from_dataframe(val_df, x_col='filepath', y_col='label',
                                                target_size=image_size, batch_size=batch_size,
                                                color_mode='rgb', class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(test_df, x_col='filepath', y_col='label',
                                                  target_size=image_size, batch_size=batch_size,
                                                  color_mode='rgb', class_mode='categorical', shuffle=False)

# Transfer Learning with MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopper = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('model_checkpoint.keras', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# Training
epochs = 30
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[early_stopper, checkpoint, reduce_lr]
)

# Save training history
with open('training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)


Found 6282 validated image filenames belonging to 7 classes.
Found 2095 validated image filenames belonging to 7 classes.
Found 2095 validated image filenames belonging to 7 classes.
9406464/9406464 [==============================] - 1s 0us/step
Epoch 1/30
196/196 [==============================] - 10s 46ms/step - loss: 1.9677 - accuracy: 0.2661 - val_loss: 1.7489 - val_accuracy: 0.3072 - lr: 0.0010
Epoch 2/30
196/196 [==============================] - 9s 47ms/step - loss: 1.7304 - accuracy: 0.3162 - val_loss: 1.6840 - val_accuracy: 0.3322 - lr: 0.0010
Epoch 3/30
196/196 [==============================] - 9s 48ms/step - loss: 1.6991 - accuracy: 0.3253 - val_loss: 1.6768 - val_accuracy: 0.3332 - lr: 0.0010
Epoch 4/30
196/196 [==============================] - 9s 48ms/step - loss: 1.6793 - accuracy: 0.3392 - val_loss: 1.6569 - val_accuracy: 0.3322 - lr: 0.0010
Epoch 5/30
196/196 [==============================] - 10s 48ms/step - loss: 1.6597 - accuracy: 0.3390 - val_loss: 1.7115 - val_ac